In [34]:
#Import dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import datetime as dt
from datetime import datetime
import os
import csv
from sqlalchemy import create_engine
from requests.api import head
from sqlalchemy.dialects.mysql import insert



# Water Usage Data Cleaning.

In [35]:
#Read dataframe.
water_county_df=pd.read_csv("Resources/Water Usage by County 2015.csv",header=0)
water_county_df

,STATE,STATEFIPS,COUNTY,COUNTYFIPS,FIPS,YEAR,TP-TotPop,PS-GWPop,PS-SWPop,PS-TOPop,...,TO-WGWTo,TO-WSWFr,TO-WSWSa,TO-WSWTo,TO-WFrTo,TO-WSaTo,TO-Wtotl,TO-CUsFrPartial,TO-CUsSaPartial,TO-CUTotPartial
0,AL,1,Autauga County,1,1001,2015,55.35,--,--,49.00,...,10.69,45.72,0.00,45.72,56.41,0.00,56.41,13.18,0.00,13.18
1,AL,1,Baldwin County,3,1003,2015,203.71,--,--,174.91,...,73.38,11.32,0.00,11.32,84.70,0.00,84.70,58.28,0.00,58.28
2,AL,1,Barbour County,5,1005,2015,26.49,--,--,24.59,...,6.06,2.96,0.00,2.96,9.02,0.00,9.02,2.96,0.00,2.96
3,AL,1,Bibb County,7,1007,2015,22.58,--,--,20.95,...,5.67,0.20,0.00,0.20,5.87,0.00,5.87,0.19,0.00,0.19
4,AL,1,Blount County,9,1009,2015,57.67,--,--,44.82,...,4.64,55.43,0.00,55.43,60.07,0.00,60.07,1.08,0.00,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,PR,72,Yabucoa Municipio,151,72151,2015,35.08,--,--,35.08,...,5.85,2.19,0.00,2.19,8.04,0.00,8.04,3.90,0.00,3.90
3219,PR,72,Yauco Municipio,153,72153,2015,37.68,--,--,37.68,...,2.26,4.22,0.00,4.22,6.48,0.00,6.48,2.00,0.00,2.00
3220,VI,78,St. Croix County,10,78010,2015,49.25,11.77,16.14,27.91,...,2.19,0.83,39.68,40.51,3.02,39.68,42.70,0.00,1.55,1.55
3221,VI,78,St. John County,20,78020,2015,4.06,0.14,0.78,0.92,...,0.08,0.12,0.06,0.18,0.20,0.06,0.26,0.00,0.00,0.00


In [36]:
#checking columns names.
water_county_df.columns

Index(['STATE', 'STATEFIPS', 'COUNTY', 'COUNTYFIPS', 'FIPS', 'YEAR',
       'TP-TotPop', 'PS-GWPop', 'PS-SWPop', 'PS-TOPop',
       ...
       'TO-WGWTo', 'TO-WSWFr', 'TO-WSWSa', 'TO-WSWTo', 'TO-WFrTo', 'TO-WSaTo',
       'TO-Wtotl', 'TO-CUsFrPartial', 'TO-CUsSaPartial', 'TO-CUTotPartial'],
      dtype='object', length=141)

In [37]:
# Dropping columns that would not be used in analysis.
water_county_dropped_col=water_county_df.drop(water_county_df.columns[[7,8,10,11,12,13,14,15,16,17,19,20,21,23,24,25,27,28,30,31,33,34,36,37,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,67,69,70,72,73,75,76,78,79,81,82,84,85,87,88,90,91,93,94,95,96,98,99,100,102,103,105,106,108,109,110,111,113,114,115,117,118,119,120,121,123,124,125,126,128,129,130,132,133,135,136,138,139]], axis = 1)
water_county_dropped_col

,STATE,STATEFIPS,COUNTY,COUNTYFIPS,FIPS,YEAR,TP-TotPop,PS-TOPop,PS-Wtotl,DO-WFrTo,...,PO-WSWTo,PO-Wtotl,PO-CUTot,PC-WGWTo,PC-Wtotl,PC-CUTot,TO-WGWTo,TO-WSWTo,TO-Wtotl,TO-CUTotPartial
0,AL,1,Autauga County,1,1001,2015,55.35,49.00,3.64,0.38,...,0.00,0.00,0.00,0.0,11.86,9.66,10.69,45.72,56.41,13.18
1,AL,1,Baldwin County,3,1003,2015,203.71,174.91,23.67,2.14,...,0.00,0.00,0.00,0.0,0.00,0.00,73.38,11.32,84.70,58.28
2,AL,1,Barbour County,5,1005,2015,26.49,24.59,3.23,0.12,...,0.00,0.00,0.00,0.0,0.00,0.00,6.06,2.96,9.02,2.96
3,AL,1,Bibb County,7,1007,2015,22.58,20.95,5.18,0.16,...,0.00,0.00,0.00,0.0,0.00,0.00,5.67,0.20,5.87,0.19
4,AL,1,Blount County,9,1009,2015,57.67,44.82,56.86,0.86,...,0.00,0.00,0.00,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,PR,72,Yabucoa Municipio,151,72151,2015,35.08,35.08,3.73,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,5.85,2.19,8.04,3.90
3219,PR,72,Yauco Municipio,153,72153,2015,37.68,37.68,4.20,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,2.26,4.22,6.48,2.00
3220,VI,78,St. Croix County,10,78010,2015,49.25,27.91,2.13,1.75,...,38.37,38.37,1.55,0.0,0.00,0.00,2.19,40.51,42.70,1.55
3221,VI,78,St. John County,20,78020,2015,4.06,0.92,0.07,0.17,...,0.00,0.00,0.00,0.0,0.00,0.00,0.08,0.18,0.26,0.00


In [38]:
#Check if column name have been updated.
water_county_dropped_col.columns

Index(['STATE', 'STATEFIPS', 'COUNTY', 'COUNTYFIPS', 'FIPS', 'YEAR',
       'TP-TotPop', 'PS-TOPop', 'PS-Wtotl', 'DO-WFrTo', 'DO-WDelv ',
       'IN-WGWTo', 'IN-WSWTo', 'IN-Wtotl', 'IR-WFrTo', 'IR-IrTot', 'LI-WFrTo',
       'AQ-WGWTo', 'AQ-WSWTo', 'AQ-Wtotl', 'MI-WGWTo', 'MI-WSWTo', 'MI-Wtotl',
       'PT-WGWTo', 'PT-WSWTo', 'PT-Wtotl', 'PT-CUTot', 'PO-WGWTo', 'PO-WSWTo',
       'PO-Wtotl', 'PO-CUTot', 'PC-WGWTo', 'PC-Wtotl', 'PC-CUTot', 'TO-WGWTo',
       'TO-WSWTo', 'TO-Wtotl', 'TO-CUTotPartial'],
      dtype='object')

In [39]:
#Dropping rows PR & VI using Index.
data_index=water_county_dropped_col.set_index("STATE")
data_index=data_index.drop(["PR","VI"], axis=0)
data_index

,STATEFIPS,COUNTY,COUNTYFIPS,FIPS,YEAR,TP-TotPop,PS-TOPop,PS-Wtotl,DO-WFrTo,DO-WDelv,...,PO-WSWTo,PO-Wtotl,PO-CUTot,PC-WGWTo,PC-Wtotl,PC-CUTot,TO-WGWTo,TO-WSWTo,TO-Wtotl,TO-CUTotPartial
STATE,,,,,,,,,,,,,,,,,,,,,
AL,1,Autauga County,1,1001,2015,55.35,49.00,3.64,0.38,3.09,...,0.0,0.0,0.0,0.0,11.86,9.66,10.69,45.72,56.41,13.18
AL,1,Baldwin County,3,1003,2015,203.71,174.91,23.67,2.14,14.92,...,0.0,0.0,0.0,0.0,0.00,0.00,73.38,11.32,84.70,58.28
AL,1,Barbour County,5,1005,2015,26.49,24.59,3.23,0.12,1.54,...,0.0,0.0,0.0,0.0,0.00,0.00,6.06,2.96,9.02,2.96
AL,1,Bibb County,7,1007,2015,22.58,20.95,5.18,0.16,1.97,...,0.0,0.0,0.0,0.0,0.00,0.00,5.67,0.20,5.87,0.19
AL,1,Blount County,9,1009,2015,57.67,44.82,56.86,0.86,3.65,...,0.0,0.0,0.0,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,56,Sweetwater County,37,56037,2015,44.63,44.23,11.37,0.03,8.57,...,0.0,0.0,0.0,0.0,19.75,14.11,19.83,132.13,151.96,48.19
WY,56,Teton County,39,56039,2015,23.13,19.94,7.78,0.24,6.17,...,0.0,0.0,0.0,0.0,0.00,0.00,7.97,103.55,111.52,22.57
WY,56,Uinta County,41,56041,2015,20.82,17.14,3.90,0.28,3.21,...,0.0,0.0,0.0,0.0,0.00,0.00,41.17,346.73,387.90,111.64


In [40]:
#Resetting State as a column.
water_county_dropped_rows=data_index.reset_index()
water_county_dropped_rows

,STATE,STATEFIPS,COUNTY,COUNTYFIPS,FIPS,YEAR,TP-TotPop,PS-TOPop,PS-Wtotl,DO-WFrTo,...,PO-WSWTo,PO-Wtotl,PO-CUTot,PC-WGWTo,PC-Wtotl,PC-CUTot,TO-WGWTo,TO-WSWTo,TO-Wtotl,TO-CUTotPartial
0,AL,1,Autauga County,1,1001,2015,55.35,49.00,3.64,0.38,...,0.0,0.0,0.0,0.0,11.86,9.66,10.69,45.72,56.41,13.18
1,AL,1,Baldwin County,3,1003,2015,203.71,174.91,23.67,2.14,...,0.0,0.0,0.0,0.0,0.00,0.00,73.38,11.32,84.70,58.28
2,AL,1,Barbour County,5,1005,2015,26.49,24.59,3.23,0.12,...,0.0,0.0,0.0,0.0,0.00,0.00,6.06,2.96,9.02,2.96
3,AL,1,Bibb County,7,1007,2015,22.58,20.95,5.18,0.16,...,0.0,0.0,0.0,0.0,0.00,0.00,5.67,0.20,5.87,0.19
4,AL,1,Blount County,9,1009,2015,57.67,44.82,56.86,0.86,...,0.0,0.0,0.0,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,56,Sweetwater County,37,56037,2015,44.63,44.23,11.37,0.03,...,0.0,0.0,0.0,0.0,19.75,14.11,19.83,132.13,151.96,48.19
3138,WY,56,Teton County,39,56039,2015,23.13,19.94,7.78,0.24,...,0.0,0.0,0.0,0.0,0.00,0.00,7.97,103.55,111.52,22.57
3139,WY,56,Uinta County,41,56041,2015,20.82,17.14,3.90,0.28,...,0.0,0.0,0.0,0.0,0.00,0.00,41.17,346.73,387.90,111.64
3140,WY,56,Washakie County,43,56043,2015,8.33,7.01,0.09,0.10,...,0.0,0.0,0.0,0.0,0.00,0.00,8.55,260.10,268.65,64.22


In [41]:
#Renaming Columns.
water_county_renamed_col=water_county_dropped_rows.rename(columns={'STATE':'State','STATEFIPS':'Statefips','COUNTY':'County','COUNTYFIPS':'Countyfips','FIPS':'Fips','YEAR':'Year','TP-TotPop':'Total population of area',\
                                                                  'PS-TOPop':'Public Supply total population served','PS-Wtotl':'Public supply total withdrawals','DO-WFrTo':'Domestic total self-supplied withdrawals','DO-WDelv':'Domestic total use','IN-WGWTo':'Industrial self-supplied groundwater withdrawals total',\
                                                                  'IN-WSWTo':'Industrial self-supplied surface-water withdrawals total','IN-Wtotl':'Industrial self-supplied total withdrawals total','IR-WFrTo':'Irrigation total withdrawals fresh','IR-IrTot':'Irrigation acres irrigated total',\
                                                                  'LI-WFrTo':'Livestock total withdrawals fresh','AQ-WGWTo':'Aquaculture groundwater withdrawals total','AQ-WSWTo':'Aquaculture surface-water withdrawals total','AQ-Wtotl':'Aquaculture total withdrawals total','MI-WGWTo':'Mining groundwater withdrawals total','MI-WSWTo':'Mining surface-water withdrawals total','MI-Wtotl':'Mining total withdrawals total',\
                                                                  'PT-WGWTo':'Thermoelectric groundwater withdrawals total','PT-WSWTo':'Thermoelectric surface-water withdrawals total','PT-Wtotl':'Thermoelectric total withdrawals total','PT-CUTot':'Thermoelectri total consumptive use total','PO-WGWTo':'Thermoelectric once-through groundwater withdrawals total','PO-WSWTo':'Thermoelectric once-through surface-water withdrawals total','PO-Wtotl':'Thermoelectric once-through total withdrawals',\
                                                                  'PO-CUTot':'Thermoelectric once-through total consumptive use total','PC-WGWTo':'Thermoelectric recirculating groundwater withdrawals total','PC-Wtotl':'Thermoelectric recirculating total withdrawals total','PC-CUTot':'Thermoelectric recirculating total consumptive use total','TO-WGWTo':'Total groundwater withdrawals total','TO-WSWTo':'Total surface-water withdrawals total','TO-Wtotl':'Total withdrawals total','TO-CUTotPartial':'Irrigation and Thermoelectric total consumptive use total'})
water_county_renamed_col


,State,Statefips,County,Countyfips,Fips,Year,Total population of area,Public Supply total population served,Public supply total withdrawals,Domestic total self-supplied withdrawals,...,Thermoelectric once-through surface-water withdrawals total,Thermoelectric once-through total withdrawals,Thermoelectric once-through total consumptive use total,Thermoelectric recirculating groundwater withdrawals total,Thermoelectric recirculating total withdrawals total,Thermoelectric recirculating total consumptive use total,Total groundwater withdrawals total,Total surface-water withdrawals total,Total withdrawals total,Irrigation and Thermoelectric total consumptive use total
0,AL,1,Autauga County,1,1001,2015,55.35,49.00,3.64,0.38,...,0.0,0.0,0.0,0.0,11.86,9.66,10.69,45.72,56.41,13.18
1,AL,1,Baldwin County,3,1003,2015,203.71,174.91,23.67,2.14,...,0.0,0.0,0.0,0.0,0.00,0.00,73.38,11.32,84.70,58.28
2,AL,1,Barbour County,5,1005,2015,26.49,24.59,3.23,0.12,...,0.0,0.0,0.0,0.0,0.00,0.00,6.06,2.96,9.02,2.96
3,AL,1,Bibb County,7,1007,2015,22.58,20.95,5.18,0.16,...,0.0,0.0,0.0,0.0,0.00,0.00,5.67,0.20,5.87,0.19
4,AL,1,Blount County,9,1009,2015,57.67,44.82,56.86,0.86,...,0.0,0.0,0.0,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,56,Sweetwater County,37,56037,2015,44.63,44.23,11.37,0.03,...,0.0,0.0,0.0,0.0,19.75,14.11,19.83,132.13,151.96,48.19
3138,WY,56,Teton County,39,56039,2015,23.13,19.94,7.78,0.24,...,0.0,0.0,0.0,0.0,0.00,0.00,7.97,103.55,111.52,22.57
3139,WY,56,Uinta County,41,56041,2015,20.82,17.14,3.90,0.28,...,0.0,0.0,0.0,0.0,0.00,0.00,41.17,346.73,387.90,111.64
3140,WY,56,Washakie County,43,56043,2015,8.33,7.01,0.09,0.10,...,0.0,0.0,0.0,0.0,0.00,0.00,8.55,260.10,268.65,64.22


In [42]:
water_county_renamed_col['Fips'] = water_county_renamed_col['Fips'].astype(str).str.pad(5, side='left', fillchar='0')
water_county_renamed_col

,State,Statefips,County,Countyfips,Fips,Year,Total population of area,Public Supply total population served,Public supply total withdrawals,Domestic total self-supplied withdrawals,...,Thermoelectric once-through surface-water withdrawals total,Thermoelectric once-through total withdrawals,Thermoelectric once-through total consumptive use total,Thermoelectric recirculating groundwater withdrawals total,Thermoelectric recirculating total withdrawals total,Thermoelectric recirculating total consumptive use total,Total groundwater withdrawals total,Total surface-water withdrawals total,Total withdrawals total,Irrigation and Thermoelectric total consumptive use total
0,AL,1,Autauga County,1,01001,2015,55.35,49.00,3.64,0.38,...,0.0,0.0,0.0,0.0,11.86,9.66,10.69,45.72,56.41,13.18
1,AL,1,Baldwin County,3,01003,2015,203.71,174.91,23.67,2.14,...,0.0,0.0,0.0,0.0,0.00,0.00,73.38,11.32,84.70,58.28
2,AL,1,Barbour County,5,01005,2015,26.49,24.59,3.23,0.12,...,0.0,0.0,0.0,0.0,0.00,0.00,6.06,2.96,9.02,2.96
3,AL,1,Bibb County,7,01007,2015,22.58,20.95,5.18,0.16,...,0.0,0.0,0.0,0.0,0.00,0.00,5.67,0.20,5.87,0.19
4,AL,1,Blount County,9,01009,2015,57.67,44.82,56.86,0.86,...,0.0,0.0,0.0,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,56,Sweetwater County,37,56037,2015,44.63,44.23,11.37,0.03,...,0.0,0.0,0.0,0.0,19.75,14.11,19.83,132.13,151.96,48.19
3138,WY,56,Teton County,39,56039,2015,23.13,19.94,7.78,0.24,...,0.0,0.0,0.0,0.0,0.00,0.00,7.97,103.55,111.52,22.57
3139,WY,56,Uinta County,41,56041,2015,20.82,17.14,3.90,0.28,...,0.0,0.0,0.0,0.0,0.00,0.00,41.17,346.73,387.90,111.64
3140,WY,56,Washakie County,43,56043,2015,8.33,7.01,0.09,0.10,...,0.0,0.0,0.0,0.0,0.00,0.00,8.55,260.10,268.65,64.22


In [43]:
#Check if column name have been updated.
water_county_renamed_col.columns

Index(['State', 'Statefips', 'County', 'Countyfips', 'Fips', 'Year',
       'Total population of area', 'Public Supply total population served',
       'Public supply total withdrawals',
       'Domestic total self-supplied withdrawals', 'DO-WDelv ',
       'Industrial self-supplied groundwater withdrawals total',
       'Industrial self-supplied surface-water withdrawals total',
       'Industrial self-supplied total withdrawals total',
       'Irrigation total withdrawals fresh',
       'Irrigation acres irrigated total', 'Livestock total withdrawals fresh',
       'Aquaculture groundwater withdrawals total',
       'Aquaculture surface-water withdrawals total',
       'Aquaculture total withdrawals total',
       'Mining groundwater withdrawals total',
       'Mining surface-water withdrawals total',
       'Mining total withdrawals total',
       'Thermoelectric groundwater withdrawals total',
       'Thermoelectric surface-water withdrawals total',
       'Thermoelectric total wit

In [44]:
# Examine data type of each columns.
water_county_renamed_col.dtypes

State                                                           object
Statefips                                                        int64
County                                                          object
Countyfips                                                       int64
Fips                                                            object
Year                                                             int64
Total population of area                                       float64
Public Supply total population served                          float64
Public supply total withdrawals                                float64
Domestic total self-supplied withdrawals                       float64
DO-WDelv                                                       float64
Industrial self-supplied groundwater withdrawals total         float64
Industrial self-supplied surface-water withdrawals total       float64
Industrial self-supplied total withdrawals total               float64
Irriga

In [45]:
# checking missing values.
water_county_renamed_col.isnull().sum()

State                                                          0
Statefips                                                      0
County                                                         0
Countyfips                                                     0
Fips                                                           0
Year                                                           0
Total population of area                                       0
Public Supply total population served                          0
Public supply total withdrawals                                0
Domestic total self-supplied withdrawals                       0
DO-WDelv                                                       0
Industrial self-supplied groundwater withdrawals total         0
Industrial self-supplied surface-water withdrawals total       0
Industrial self-supplied total withdrawals total               0
Irrigation total withdrawals fresh                             0
Irrigation acres irrigate

In [46]:
## checking null values.
water_county_renamed_col.dropna()

,State,Statefips,County,Countyfips,Fips,Year,Total population of area,Public Supply total population served,Public supply total withdrawals,Domestic total self-supplied withdrawals,...,Thermoelectric once-through surface-water withdrawals total,Thermoelectric once-through total withdrawals,Thermoelectric once-through total consumptive use total,Thermoelectric recirculating groundwater withdrawals total,Thermoelectric recirculating total withdrawals total,Thermoelectric recirculating total consumptive use total,Total groundwater withdrawals total,Total surface-water withdrawals total,Total withdrawals total,Irrigation and Thermoelectric total consumptive use total
0,AL,1,Autauga County,1,01001,2015,55.35,49.00,3.64,0.38,...,0.0,0.0,0.0,0.0,11.86,9.66,10.69,45.72,56.41,13.18
1,AL,1,Baldwin County,3,01003,2015,203.71,174.91,23.67,2.14,...,0.0,0.0,0.0,0.0,0.00,0.00,73.38,11.32,84.70,58.28
2,AL,1,Barbour County,5,01005,2015,26.49,24.59,3.23,0.12,...,0.0,0.0,0.0,0.0,0.00,0.00,6.06,2.96,9.02,2.96
3,AL,1,Bibb County,7,01007,2015,22.58,20.95,5.18,0.16,...,0.0,0.0,0.0,0.0,0.00,0.00,5.67,0.20,5.87,0.19
4,AL,1,Blount County,9,01009,2015,57.67,44.82,56.86,0.86,...,0.0,0.0,0.0,0.0,0.00,0.00,4.64,55.43,60.07,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,56,Sweetwater County,37,56037,2015,44.63,44.23,11.37,0.03,...,0.0,0.0,0.0,0.0,19.75,14.11,19.83,132.13,151.96,48.19
3138,WY,56,Teton County,39,56039,2015,23.13,19.94,7.78,0.24,...,0.0,0.0,0.0,0.0,0.00,0.00,7.97,103.55,111.52,22.57
3139,WY,56,Uinta County,41,56041,2015,20.82,17.14,3.90,0.28,...,0.0,0.0,0.0,0.0,0.00,0.00,41.17,346.73,387.90,111.64
3140,WY,56,Washakie County,43,56043,2015,8.33,7.01,0.09,0.10,...,0.0,0.0,0.0,0.0,0.00,0.00,8.55,260.10,268.65,64.22


# Drought  Data Cleaning

In [47]:
#Read dataframe.
drought_county_df=pd.read_csv("Resources/Drought by county from 20150101_20151231.csv")
drought_county_df

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20151229,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/29/2015,1/4/2016,1
1,20151222,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/22/2015,12/28/2015,1
2,20151215,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/15/2015,12/21/2015,1
3,20151208,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/8/2015,12/14/2015,1
4,20151201,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/1/2015,12/7/2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166574,20150127,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/27/2015,2/2/2015,1
166575,20150120,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/20/2015,1/26/2015,1
166576,20150113,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/13/2015,1/19/2015,1
166577,20150106,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/6/2015,1/12/2015,1


In [48]:
#checking columns names.
drought_county_df.columns

Index(['MapDate', 'FIPS', 'County', 'State', 'None', 'D0', 'D1', 'D2', 'D3',
       'D4', 'ValidStart', 'ValidEnd', 'StatisticFormatID'],
      dtype='object')

In [49]:
# Dropping columns that would not be used in analysis.
drought_county_dropped_col=drought_county_df.drop(drought_county_df.columns[[0,12]], axis = 1)
drought_county_dropped_col

,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd
0,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/29/2015,1/4/2016
1,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/22/2015,12/28/2015
2,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/15/2015,12/21/2015
3,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/8/2015,12/14/2015
4,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/1/2015,12/7/2015
...,...,...,...,...,...,...,...,...,...,...,...
166574,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/27/2015,2/2/2015
166575,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/20/2015,1/26/2015
166576,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/13/2015,1/19/2015
166577,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/6/2015,1/12/2015


In [50]:
#Renaming Columns.
drought_county_renamed_col=drought_county_dropped_col.rename(columns={'FIPS':'Fips','None':'No drought','D0':'Abnormally dry','D1':'Moderate drought','D2':'Severe drought','D3':'Extreme drought','D4':'Exceptional drought','ValidStart':'Validstart','ValidEnd':'Validend'})
drought_county_renamed_col

,Fips,County,State,No drought,Abnormally dry,Moderate drought,Severe drought,Extreme drought,Exceptional drought,Validstart,Validend
0,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/29/2015,1/4/2016
1,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/22/2015,12/28/2015
2,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/15/2015,12/21/2015
3,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/8/2015,12/14/2015
4,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/1/2015,12/7/2015
...,...,...,...,...,...,...,...,...,...,...,...
166574,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/27/2015,2/2/2015
166575,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/20/2015,1/26/2015
166576,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/13/2015,1/19/2015
166577,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/6/2015,1/12/2015


In [51]:
drought_county_renamed_col['Fips'] = drought_county_renamed_col['Fips'].astype(str).str.pad(5, side='left', fillchar='0')
drought_county_renamed_col

,Fips,County,State,No drought,Abnormally dry,Moderate drought,Severe drought,Extreme drought,Exceptional drought,Validstart,Validend
0,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/29/2015,1/4/2016
1,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/22/2015,12/28/2015
2,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/15/2015,12/21/2015
3,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/8/2015,12/14/2015
4,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/1/2015,12/7/2015
...,...,...,...,...,...,...,...,...,...,...,...
166574,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/27/2015,2/2/2015
166575,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/20/2015,1/26/2015
166576,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/13/2015,1/19/2015
166577,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/6/2015,1/12/2015


In [52]:
# checking missing values.
drought_county_renamed_col.isnull().sum()

Fips                   0
County                 0
State                  0
No drought             0
Abnormally dry         0
Moderate drought       0
Severe drought         0
Extreme drought        0
Exceptional drought    0
Validstart             0
Validend               0
dtype: int64

In [53]:
## checking null values.
drought_county_renamed_col.dropna()

,Fips,County,State,No drought,Abnormally dry,Moderate drought,Severe drought,Extreme drought,Exceptional drought,Validstart,Validend
0,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/29/2015,1/4/2016
1,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/22/2015,12/28/2015
2,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/15/2015,12/21/2015
3,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/8/2015,12/14/2015
4,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,12/1/2015,12/7/2015
...,...,...,...,...,...,...,...,...,...,...,...
166574,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/27/2015,2/2/2015
166575,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/20/2015,1/26/2015
166576,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/13/2015,1/19/2015
166577,56045,Weston County,WY,100.0,0.0,0.0,0.0,0.0,0.0,1/6/2015,1/12/2015


In [54]:
# Examine data type of each columns.
drought_county_renamed_col.dtypes

Fips                    object
County                  object
State                   object
No drought             float64
Abnormally dry         float64
Moderate drought       float64
Severe drought         float64
Extreme drought        float64
Exceptional drought    float64
Validstart              object
Validend                object
dtype: object

In [55]:
# Convert Valid start to type datetime.
drought_county_renamed_col["Validstart"]= pd.to_datetime(drought_county_renamed_col.Validstart , format='%m/%d/%Y')
drought_county_renamed_col["Validend"]= pd.to_datetime(drought_county_renamed_col.Validend , format='%m/%d/%Y')
drought_county_renamed_col.dtypes

Fips                           object
County                         object
State                          object
No drought                    float64
Abnormally dry                float64
Moderate drought              float64
Severe drought                float64
Extreme drought               float64
Exceptional drought           float64
Validstart             datetime64[ns]
Validend               datetime64[ns]
dtype: object

In [64]:

drought_county_renamed_col.head()

,Fips,County,State,No drought,Abnormally dry,Moderate drought,Severe drought,Extreme drought,Exceptional drought,Validstart,Validend
0,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2015-12-29,2016-01-04
1,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2015-12-22,2015-12-28
2,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2015-12-15,2015-12-21
3,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2015-12-08,2015-12-14
4,02013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2015-12-01,2015-12-07


In [57]:
#Finding average by county.
ave_counties_drought=drought_county_renamed_col.groupby("Fips")[["No drought","Abnormally dry","Moderate drought","Severe drought","Extreme drought","Exceptional drought"]].mean()
ave_counties_drought

,No drought,Abnormally dry,Moderate drought,Severe drought,Extreme drought,Exceptional drought
Fips,,,,,,
01001,53.090189,46.909811,0.000000,0.000000,0.0,0.0
01003,38.361698,61.638302,28.854528,0.956415,0.0,0.0
01005,54.769245,45.230755,4.709057,0.000000,0.0,0.0
01007,77.398113,22.601887,1.642830,0.000000,0.0,0.0
01009,62.085849,37.914151,10.798491,0.000000,0.0,0.0
...,...,...,...,...,...,...
56037,34.760000,65.240000,17.692264,1.299623,0.0,0.0
56039,38.224717,61.775283,0.577170,0.000000,0.0,0.0
56041,13.390943,86.609057,22.962830,0.000000,0.0,0.0


# Connect to database

In [58]:
# create CSV's inform a database by drought.
ave_counties_drought.to_csv('database/droughtdata.csv', index=False)

In [59]:
# create CSV's inform a database by water.
water_county_renamed_col.to_csv('database/waterdata.csv', index=False)

In [65]:
drought_county_renamed_col.to_csv('database/drought_full.csv', index=False)

<!-- Loading dataframes into a SQLlite database -->

In [66]:
#Importdependancy.
import sqlalchemy as db
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy import select

In [67]:
#create engine.
engine = create_engine('sqlite:///database/drought_water.db', echo=False)

In [69]:
from sqlalchemy.exc import IntegrityError

try:
    ave_counties_drought.to_sql(name='average_drought', con=engine, if_exists='replace', index=True)
    water_county_renamed_col.to_sql(name='water', con=engine, if_exists='replace', index=True)
    drought_county_renamed_col.to_sql(name='drought', con=engine, if_exists='replace', index=True)
    
    # engine.execute(mytable.insert(), **kwargs)
except IntegrityError:
    pass



In [70]:
#Check for tables
engine.table_names()

c:\Users\kmpfl\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['average_drought', 'drought', 'test', 'water']

In [74]:
sql = 'DROP TABLE IF EXISTS test;'
result = engine.execute(sql)
engine.table_names()

c:\Users\kmpfl\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  This is separate from the ipykernel package so we can avoid doing imports until


['average_drought', 'drought', 'water']

In [71]:
print (len(['drought']))


1
